In [ ]:
import math


class EuclideanDistTracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 25:
                    self.center_points[id] = (cx, cy)
                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids


In [ ]:
import cv2
from tracker import *

### Create Tracker object

In [ ]:
tracker = EuclideanDistTracker()

In [ ]:
cap = cv2.VideoCapture(r'D:\7.30 PM DATA SCIENCE AND AI\TF, KERAS, OPENCV CODES\sample_videos\highway.mp4')

In [ ]:
# Object Detection from Stable Camera
object_detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=40)

while True:
    ret, frame = cap.read()
    height, width, _ = frame.shape
    
    # Extract Region of Interest (RoI)
    roi = frame[340:720, 500:800]  #  extract a portion of the frame from row 340 to 720 and column 500 to 800
    
    # 1. Object Detection
    mask = object_detector.apply(roi)
    
    # Threshold the mask
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    # Pixels with values greater than 254 are set to 255 (white), and the rest are set to 0 (black)
    # Converts the mask into a binary image, where white pixels represent the detected objects, and black pixels represent the background
    
    
    # Contours are the boundaries of the white regions in the binary image
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    detections = []
    
    for cnt in contours:
        # Calculate area and remove small elements
        area = cv2.contourArea(cnt)
        
        if area > 100:
            # Calculate bounding rectangle : minimal area rectangle that encloses the contour
            x, y, w, h = cv2.boundingRect(cnt)
            detections.append([x,y,w,h])
            
    # 2. Object Tracking
    boxes_ids = tracker.update(detections)
    
    # For each tracked object, a bounding rectangle is drawn around it, and its unique ID is displayed above the rectangle
    # Visualize the tracked objects in the Region of Interest (RoI).
    for box_id in boxes_ids:
        x,y,w,h, id = box_id
        cv2.putText(roi, str(id), (x, y-15), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255), 2)
        cv2.rectangle(roi, (x,y), (x+w, y+h), (0,255,0), 3)
        
        cv2.imshow('ROI', roi)
        cv2.imshow('Frame', frame)
        cv2.imshow('Mask', mask)
        
        if cv2.waitKey(1)==13:
            break
            
cap.release()
cv2.destroyAllWindows()